In [1]:
import os
import json
import boto3
from typing import List, Tuple, Dict

import mlflow
import torch
import numpy as np
from mlflow.tracking import MlflowClient
from mlflow.store.artifact.s3_artifact_repo import S3ArtifactRepository

# === Cấu hình MLflow + MinIO ===
def configure_mlflow(s3_endpoint: str, aws_key: str, aws_secret: str, tracking_uri: str):
    os.environ.update({
        "AWS_ACCESS_KEY_ID": aws_key,
        "AWS_SECRET_ACCESS_KEY": aws_secret,
        "MLFLOW_S3_ENDPOINT_URL": s3_endpoint,
        "MLFLOW_S3_IGNORE_TLS": "true"
    })
    mlflow.set_tracking_uri(tracking_uri)

    session = boto3.session.Session(
        aws_access_key_id=aws_key,
        aws_secret_access_key=aws_secret
    )
    S3ArtifactRepository._get_s3_client = lambda self: session.client("s3", endpoint_url=s3_endpoint)

# === IDMapper đơn giản ===
class SimpleIDMapper:
    def __init__(self, mapping_path: str):
        with open(mapping_path, "r") as f:
            mapping = json.load(f)
        self.item_to_index: Dict[str, int] = mapping["item_to_index"]
        self.index_to_item: Dict[int, str] = mapping["index_to_item"]

# === Load TorchScript model + IDMapper từ registered model ===
def load_champion_model(model_name: str) -> Tuple[torch.jit.ScriptModule, SimpleIDMapper]:
    client = MlflowClient()
    versions = client.search_model_versions(f"name='{model_name}'")
    champs = [v for v in versions if v.tags.get("champion", "").lower() == "true"]
    if not champs:
        raise RuntimeError(f"❌ No champion version found for model '{model_name}'")

    champ = max(champs, key=lambda v: v.creation_timestamp)
    run_id = champ.run_id
    model = mlflow.pytorch.load_model(f"models:/{model_name}/{champ.version}")
    model.eval()

    id_mapper_path = f"runs:/{run_id}/id_mapper/id_mapper.json"
    id_mapper_local = mlflow.artifacts.download_artifacts(id_mapper_path)
    id_mapper = SimpleIDMapper(id_mapper_local)

    print(f"✅ Loaded model v{champ.version} (run_id={run_id}) and IDMapper.")
    return model, id_mapper

# === Inference Utilities ===
def get_item_embedding(model, id_mapper, item_id, device=torch.device("cpu")) -> torch.Tensor:
    idx = torch.tensor([id_mapper.item_to_index[item_id]], device=device)
    return model.embeddings(idx).squeeze(0)

def get_topk_similar(model, id_mapper, item_id, top_k=10, device=torch.device("cpu")) -> List[Tuple[str, int, float]]:
    target_idx = id_mapper.item_to_index[item_id]
    weight = model.embeddings.weight[:-1].to(device)  # exclude padding_idx
    with torch.no_grad():
        target_emb = weight[target_idx]
        sims = torch.matmul(weight, target_emb)
    sims[target_idx] = -np.inf  # exclude self
    topk = torch.topk(sims, k=top_k)
    return [
        (id_mapper.index_to_item[int(i)], int(i), float(s))
        for i, s in zip(topk.indices, topk.values)
    ]

def predict_batch(model, id_mapper, batch: Dict[str, List[str]], device=torch.device("cpu")) -> np.ndarray:
    tgt = torch.tensor([id_mapper.item_to_index[i] for i in batch["target_items"]], device=device)
    ctx = torch.tensor([id_mapper.item_to_index[i] for i in batch["context_items"]], device=device)
    with torch.no_grad():
        return model(tgt, ctx).cpu().numpy()

# === Chạy thử ===
if __name__ == "__main__":
    # 1. Cấu hình
    configure_mlflow(
        s3_endpoint="http://127.0.0.1:9010",
        aws_key="admin",
        aws_secret="Password1234",
        tracking_uri="http://localhost:5002"
    )

    # 2. Load model + IDMapper
    model_name = "item2vec_skipgram"
    model, id_mapper = load_champion_model(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 3. In embedding
    test_item = "B00000IV95"
    test_idx = id_mapper.item_to_index[test_item]
    emb = get_item_embedding(model, id_mapper, test_item, device)
    print(f"\n🔹 Embedding for '{test_item}' (index {test_idx}): {emb[:5].detach().cpu().numpy()}")

    # 4. Tìm top K tương tự
    topk = get_topk_similar(model, id_mapper, test_item, top_k=5, device=device)
    print(f"\n🔹 Top 5 similar items to '{test_item}':")
    for iid, idx, score in topk:
        print(f" - {iid} (index {idx}): score = {score:.4f}")

    # 5. Dự đoán batch
    batch = {
        "target_items": [test_item, "B0792X1RSC"],
        "context_items": ["B0792X1RSC", test_item]
    }
    scores = predict_batch(model, id_mapper, batch, device)
    print(f"\n🔹 Batch prediction scores:")
    for t, c, s in zip(batch["target_items"], batch["context_items"], scores):
        print(f" - ({t}, {c}): score = {s:.4f}")


2025/06/29 08:56:40 WARNING mlflow.pytorch: Stored model version '2.6.0+cu124' does not match installed PyTorch version '2.7.1+cu126'
/home/duong/Documents/datn1/.venv/lib/python3.11/site-packages/torch/serialization.py:1488: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


✅ Loaded model v1 (run_id=aac89aa42a3c447eac1294546df933e6) and IDMapper.

🔹 Embedding for 'B00000IV95' (index 36): [ 0.05149313  0.0443965  -0.01442701  0.07874709  0.07239346]

🔹 Top 5 similar items to 'B00000IV95':
 - B0C3H818H4 (index 4009): score = 2.2501
 - B0C48KPLZ2 (index 4026): score = 1.6529
 - 0975277324 (index 5): score = 1.6294
 - B0C1FX3BGK (index 3976): score = 1.5719
 - B00000IV35 (index 35): score = 1.5157

🔹 Batch prediction scores:
 - (B00000IV95, B0792X1RSC): score = 0.5309
 - (B0792X1RSC, B00000IV95): score = 0.5309


## Load embedding to Qdrant

In [2]:
import os
import sys
import json
import mlflow
import torch
import numpy as np
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from qdrant_client.http.exceptions import UnexpectedResponse
from mlflow.tracking import MlflowClient
from mlflow.store.artifact.s3_artifact_repo import S3ArtifactRepository
import boto3

# === CONFIGURATION ===
PROJECT_ROOT = "/home/duong/Documents/datn1/src/model_item2vec"
sys.path.append(PROJECT_ROOT)

load_dotenv()

# MLflow config
S3_ENDPOINT = "http://127.0.0.1:9010"
AWS_KEY = "admin"
AWS_SECRET = "Password1234"
TRACKING_URI = "http://localhost:5002"

os.environ["AWS_ACCESS_KEY_ID"] = AWS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET
os.environ["MLFLOW_S3_ENDPOINT_URL"] = S3_ENDPOINT
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"

mlflow.set_tracking_uri(TRACKING_URI)

# Configure boto3 for MinIO
session = boto3.session.Session(
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)
S3ArtifactRepository._get_s3_client = lambda self: session.client("s3", endpoint_url=S3_ENDPOINT)

# Qdrant config
QDRANT_URL = os.getenv("QDRANT_URL", "http://localhost:6333")
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION_NAME", "item2vec_collection")

MODEL_NAME = "item2vec_skipgram"
TAG_NAME = "champion"

# === SimpleIDMapper ===
class SimpleIDMapper:
    def __init__(self, mapping_path: str):
        with open(mapping_path, "r") as f:
            mapping = json.load(f)
        self.item_to_index = mapping["item_to_index"]
        self.index_to_item = mapping["index_to_item"]

# === STEP 1: LOAD MODEL AND ID MAPPING ===
def load_model_from_mlflow(model_name: str, tag: str = "champion") -> tuple:
    client = MlflowClient()
    versions = client.search_model_versions(f"name='{model_name}'")
    champs = [v for v in versions if v.tags.get(tag, "").lower() == "true"]
    if not champs:
        raise ValueError(f"❌ No version tagged '{tag}' found for model '{model_name}'.")

    champ = max(champs, key=lambda v: v.creation_timestamp)
    run_id = champ.run_id
    model_uri = f"models:/{model_name}/{champ.version}"
    print(f"📦 Loading model from URI: {model_uri}")
    model = mlflow.pytorch.load_model(model_uri)
    model.eval()

    id_mapper_path = f"runs:/{run_id}/id_mapper/id_mapper.json"
    id_mapper_local = mlflow.artifacts.download_artifacts(id_mapper_path)
    id_mapper = SimpleIDMapper(id_mapper_local)

    print(f"✅ Loaded model version {champ.version} (run_id={run_id}) with {len(id_mapper.item_to_index)} items.")
    return model, id_mapper

# === STEP 2: EXTRACT EMBEDDINGS ===
def get_all_embeddings(model, id_mapper: SimpleIDMapper) -> tuple:
    item_ids = list(id_mapper.item_to_index.keys())
    item_indices = [id_mapper.item_to_index[id_] for id_ in item_ids]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    tensor_indices = torch.tensor(item_indices, device=device)
    with torch.no_grad():
        embeddings = model.embeddings(tensor_indices).detach().cpu().numpy()
    print(f"✅ Extracted embeddings with shape {embeddings.shape}")
    return item_ids, embeddings

# === STEP 3: INDEX TO QDRANT (delete and recreate collection) ===
def index_embeddings_to_qdrant(item_ids: list, embeddings: np.ndarray, qdrant_url: str, collection_name: str):
    try:
        client = QdrantClient(url=qdrant_url)
        print(f"🌐 Connecting to Qdrant at {qdrant_url}")

        vector_dim = embeddings.shape[1]

        # Delete collection if it exists
        if client.collection_exists(collection_name):
            client.delete_collection(collection_name)
            print(f"🗑️ Deleted existing collection '{collection_name}'.")

        # Create new collection
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_dim, distance=Distance.COSINE),
        )
        print(f"✅ Created new collection '{collection_name}' with dim={vector_dim}")

        # Prepare points using sequential IDs
        points = [
            PointStruct(id=idx, vector=vec.tolist(), payload={"item_id": item_id})
            for idx, (item_id, vec) in enumerate(zip(item_ids, embeddings))
        ]

        # Upsert points
        upsert_result = client.upsert(collection_name=collection_name, points=points)
        assert str(upsert_result.status) == "completed"
        print(f"✅ Upserted {len(points)} embeddings into Qdrant collection '{collection_name}'")

    except UnexpectedResponse as e:
        print("❌ Qdrant returned unexpected response (possibly 404 or 400).")
        print("➡️ Check that the endpoint is correct and Qdrant is running.")
        print(str(e))
        raise
    except Exception as e:
        print("❌ Failed to index embeddings to Qdrant.")
        print(str(e))
        raise

# === MAIN ===
def main():
    print("🚀 Starting embedding indexing...")
    model, id_mapper = load_model_from_mlflow(MODEL_NAME, TAG_NAME)
    item_ids, embeddings = get_all_embeddings(model, id_mapper)
    index_embeddings_to_qdrant(item_ids, embeddings, QDRANT_URL, QDRANT_COLLECTION)

if __name__ == "__main__":
    main()

🚀 Starting embedding indexing...
📦 Loading model from URI: models:/item2vec_skipgram/1


2025/06/29 09:00:35 WARNING mlflow.pytorch: Stored model version '2.6.0+cu124' does not match installed PyTorch version '2.7.1+cu126'


✅ Loaded model version 1 (run_id=aac89aa42a3c447eac1294546df933e6) with 4143 items.
✅ Extracted embeddings with shape (4143, 256)
🌐 Connecting to Qdrant at http://localhost:6333
✅ Created new collection 'item2vec_collection' with dim=256
✅ Upserted 4143 embeddings into Qdrant collection 'item2vec_collection'


### Test Qdrant

In [3]:
import os
import json
import torch
import mlflow
from qdrant_client import QdrantClient
from qdrant_client.http.exceptions import UnexpectedResponse
from mlflow.tracking import MlflowClient
from mlflow.store.artifact.s3_artifact_repo import S3ArtifactRepository
import boto3

# === CONFIGURATION ===
QDRANT_URL = "http://localhost:6333"
QDRANT_COLLECTION = "item2vec_collection"
MODEL_NAME = "item2vec_skipgram"
TAG_NAME = "champion"
ITEM_ID = "B00EMGM1JQ"
TOP_K = 5

S3_ENDPOINT = "http://127.0.0.1:9010"
AWS_KEY = "admin"
AWS_SECRET = "Password1234"
TRACKING_URI = "http://localhost:5002"

# Configure MLflow and MinIO
os.environ["AWS_ACCESS_KEY_ID"] = AWS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET
os.environ["MLFLOW_S3_ENDPOINT_URL"] = S3_ENDPOINT
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
mlflow.set_tracking_uri(TRACKING_URI)

session = boto3.session.Session(
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)
S3ArtifactRepository._get_s3_client = lambda self: session.client("s3", endpoint_url=S3_ENDPOINT)

# === SimpleIDMapper ===
class SimpleIDMapper:
    def __init__(self, mapping_path: str):
        with open(mapping_path, "r") as f:
            mapping = json.load(f)
        self.item_to_index = mapping["item_to_index"]
        self.index_to_item = mapping["index_to_item"]

# === STEP 1: Load model and ID mapping ===
def load_model_and_mapping(model_name: str, tag: str = "champion") -> tuple:
    client = MlflowClient()
    versions = client.search_model_versions(f"name='{model_name}'")
    champs = [v for v in versions if v.tags.get(tag, "").lower() == "true"]
    if not champs:
        raise ValueError(f"❌ No version tagged '{tag}' found for model '{model_name}'.")

    champ = max(champs, key=lambda v: v.creation_timestamp)
    run_id = champ.run_id
    model_uri = f"models:/{model_name}/{champ.version}"
    print(f"📦 Loading model from URI: {model_uri}")
    model = mlflow.pytorch.load_model(model_uri)
    model.eval()

    id_mapper_path = f"runs:/{run_id}/id_mapper/id_mapper.json"
    id_mapper_local = mlflow.artifacts.download_artifacts(id_mapper_path)
    id_mapper = SimpleIDMapper(id_mapper_local)

    print(f"✅ Loaded model version {champ.version} (run_id={run_id}) with {len(id_mapper.item_to_index)} items.")
    return model, id_mapper

# === STEP 2: Get embedding vector for ITEM_ID ===
def get_item_embedding(model, id_mapper: SimpleIDMapper, item_id: str, device: torch.device) -> np.ndarray:
    idx = id_mapper.item_to_index.get(item_id)
    if idx is None:
        raise ValueError(f"❌ Item ID '{item_id}' not found in ID mapping")
    tensor_idx = torch.tensor([idx], device=device)
    with torch.no_grad():
        embedding = model.embeddings(tensor_idx).detach().cpu().numpy()[0]
    return embedding

# === STEP 3: Query Qdrant with embedding vector ===
def search_similar_items(qdrant_url: str, collection_name: str, embedding: np.ndarray, top_k: int) -> list:
    try:
        qdrant = QdrantClient(url=qdrant_url)
        print(f"🌐 Connecting to Qdrant at {qdrant_url}")
        if not qdrant.collection_exists(collection_name):
            raise ValueError(f"❌ Collection '{collection_name}' does not exist in Qdrant")
        
        results = qdrant.search(
            collection_name=collection_name,
            query_vector=embedding.tolist(),
            limit=top_k,
        )
        return results
    except UnexpectedResponse as e:
        print("❌ Qdrant returned unexpected response (possibly 404 or 400).")
        print("➡️ Check that the endpoint is correct and Qdrant is running.")
        print(str(e))
        raise
    except Exception as e:
        print("❌ Failed to query Qdrant.")
        print(str(e))
        raise

# === MAIN ===
def main():
    print("🚀 Starting similarity search...")
    # Load model and ID mapping
    model, id_mapper = load_model_and_mapping(MODEL_NAME, TAG_NAME)
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Get embedding for ITEM_ID
    embedding = get_item_embedding(model, id_mapper, ITEM_ID, device)
    print(f"✅ Generated embedding for '{ITEM_ID}' with shape {embedding.shape}")
    
    # Query Qdrant
    results = search_similar_items(QDRANT_URL, QDRANT_COLLECTION, embedding, TOP_K)
    
    # Display results
    print(f"\n🔍 Top {TOP_K} items similar to '{ITEM_ID}':")
    for r in results:
        item = r.payload.get("item_id", "<missing>")
        print(f"🟢 {item:15} | Score: {r.score:.4f}")

if __name__ == "__main__":
    main()

🚀 Starting similarity search...
📦 Loading model from URI: models:/item2vec_skipgram/1


2025/06/29 09:00:46 WARNING mlflow.pytorch: Stored model version '2.6.0+cu124' does not match installed PyTorch version '2.7.1+cu126'


✅ Loaded model version 1 (run_id=aac89aa42a3c447eac1294546df933e6) with 4143 items.
✅ Generated embedding for 'B00EMGM1JQ' with shape (256,)
🌐 Connecting to Qdrant at http://localhost:6333

🔍 Top 5 items similar to 'B00EMGM1JQ':
🟢 B00EMGM1JQ      | Score: 1.0000
🟢 B00C6Q4HEQ      | Score: 0.7464
🟢 B00C6Q5S44      | Score: 0.6372
🟢 B00C6Q1Z6E      | Score: 0.6326
🟢 B00CQHZ2LW      | Score: 0.6079


## Load pre-recommend to Redis

In [5]:
import json
import os
import redis
import torch
import mlflow
import numpy as np
import sys
from qdrant_client import QdrantClient
from tqdm.auto import tqdm
import boto3
from mlflow.tracking import MlflowClient
from mlflow.store.artifact.s3_artifact_repo import S3ArtifactRepository

# === CONFIGURATION ===
PROJECT_ROOT = "/home/duong/Documents/datn1/src/model_item2vec"
sys.path.append(PROJECT_ROOT)

CONFIG = {
    "mlf_model_name": "item2vec_skipgram",
    "qdrant_url": f"{os.getenv('QDRANT_HOST', 'localhost')}:{os.getenv('QDRANT_PORT', '6333')}",
    "qdrant_collection_name": os.getenv("QDRANT_COLLECTION_NAME", "item2vec_collection"),
    "redis_host": os.getenv("REDIS_HOST", "localhost"),
    "redis_port": 6379,
    "redis_db": int(os.getenv("REDIS_DB", 0)),
    "batch_size": 256,
    "top_k": 10,
    "top_K": 100,
    "output_file": "../../data/batch_recs.jsonl",
    "s3_endpoint": "http://127.0.0.1:9010",
    "aws_key": "admin",
    "aws_secret": "Password1234"
}

# === ENV SETUP ===
def configure_mlflow():
    os.environ.update({
        "AWS_ACCESS_KEY_ID": CONFIG["aws_key"],
        "AWS_SECRET_ACCESS_KEY": CONFIG["aws_secret"],
        "MLFLOW_S3_ENDPOINT_URL": CONFIG["s3_endpoint"],
        "MLFLOW_S3_IGNORE_TLS": "true"
    })
    mlflow.set_tracking_uri("http://localhost:5002")

    session = boto3.session.Session(
        aws_access_key_id=CONFIG["aws_key"],
        aws_secret_access_key=CONFIG["aws_secret"]
    )
    S3ArtifactRepository._get_s3_client = lambda self: session.client("s3", endpoint_url=CONFIG["s3_endpoint"])

# === CLIENTS ===
redis_client = redis.Redis(
    host=CONFIG["redis_host"],
    port=CONFIG["redis_port"],
    db=CONFIG["redis_db"],
    decode_responses=True,
    password="123456"
)
qdrant_client = QdrantClient(url=CONFIG["qdrant_url"])

# === SimpleIDMapper ===
class SimpleIDMapper:
    def __init__(self, mapping_path: str):
        with open(mapping_path, "r") as f:
            mapping = json.load(f)
        self.item_to_index = mapping["item_to_index"]
        # Convert list to dict: index (int) -> item_id (str)
        self.index_to_item = {i: item_id for i, item_id in enumerate(mapping["index_to_item"])}
# === Load TorchScript model + IDMapper ===
def load_model():
    configure_mlflow()
    client = MlflowClient()
    versions = client.search_model_versions(f"name='{CONFIG['mlf_model_name']}'")
    champs = [v for v in versions if v.tags.get("champion", "").lower() == "true"]
    if not champs:
        raise RuntimeError(f"❌ No champion version found for model '{CONFIG['mlf_model_name']}'")

    champ = max(champs, key=lambda v: v.creation_timestamp)
    run_id = champ.run_id
    model = mlflow.pytorch.load_model(f"models:/{CONFIG['mlf_model_name']}/{champ.version}")
    model.eval()

    id_mapper_path = f"runs:/{run_id}/id_mapper/id_mapper.json"
    id_mapper_local = mlflow.artifacts.download_artifacts(id_mapper_path)
    id_mapper = SimpleIDMapper(id_mapper_local)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print(f"✅ Loaded model v{champ.version} (run_id={run_id}) and IDMapper.")
    return model, id_mapper

# === Inference Utilities ===
def get_item_embedding(model, id_mapper, item_id, device=torch.device("cpu")) -> torch.Tensor:
    idx = torch.tensor([id_mapper.item_to_index[item_id]], device=device)
    return model.embeddings(idx).squeeze(0)

def get_topk_similar(model, id_mapper, item_id, top_k=10, device=torch.device("cpu")) -> list:
    target_idx = id_mapper.item_to_index[item_id]
    weight = model.embeddings.weight[:-1].to(device)  # exclude padding_idx
    with torch.no_grad():
        target_emb = weight[target_idx]
        sims = torch.matmul(weight, target_emb)
    sims[target_idx] = -np.inf  # exclude self
    topk = torch.topk(sims, k=top_k)
    return [
        (id_mapper.index_to_item[int(i)], float(s))
        for i, s in zip(topk.indices, topk.values)
    ]

# === Compute Recommendations ===
def compute_recommendations():
    model, id_mapper = load_model()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    all_indices = list(id_mapper.index_to_item.keys())

    # Load all embeddings from Qdrant
    records = qdrant_client.retrieve(CONFIG["qdrant_collection_name"], ids=all_indices, with_vectors=True)
    id_to_vec = {rec.id: rec.vector for rec in records if rec.vector is not None}

    recs = []

    for i in tqdm(range(0, len(all_indices), CONFIG["batch_size"]), desc="Processing"):
        batch_indices = all_indices[i:i + CONFIG["batch_size"]]
        batch_vectors = [id_to_vec.get(idx) for idx in batch_indices]

        batch_neighbors = []
        for idx, vec in zip(batch_indices, batch_vectors):
            if vec is None:
                batch_neighbors.append([])
                continue
            neighbors = qdrant_client.search(
                collection_name=CONFIG["qdrant_collection_name"],
                query_vector=vec,
                limit=CONFIG["top_K"] + 1,
            )
            # Remove self from results
            neighbor_ids = [n.id for n in neighbors if n.id != idx][:CONFIG["top_K"]]
            batch_neighbors.append(neighbor_ids)

        batch_scores = []
        for idx, neighbors in zip(batch_indices, batch_neighbors):
            if not neighbors:
                batch_scores.append([])
                continue

            target_id = id_mapper.index_to_item[idx]
            neighbor_ids = [id_mapper.index_to_item[nid] for nid in neighbors]

            # Compute scores using model
            sample_input = {
                "target_items": [target_id] * len(neighbor_ids),
                "context_items": neighbor_ids
            }
            try:
                scores = predict_batch(model, id_mapper, sample_input, device)
                batch_scores.append(scores.tolist())
            except Exception as e:
                print(f"❌ Prediction failed for {target_id}: {e}")
                batch_scores.append([0.0] * len(neighbor_ids))

        # Write to Redis and JSONL
        for idx, neighbors, scores in zip(batch_indices, batch_neighbors, batch_scores):
            if not neighbors:
                continue
            sorted_pairs = sorted(zip(neighbors, scores), key=lambda x: x[1], reverse=True)
            top_neighbors, top_scores = zip(*sorted_pairs[:CONFIG["top_k"]])
            neighbor_ids = [id_mapper.index_to_item[n] for n in top_neighbors]
            target_id = id_mapper.index_to_item[idx]

            rec = {
                "target_item": target_id,
                "rec_item_ids": neighbor_ids,
                "rec_scores": list(top_scores)
            }

            recs.append(rec)
            redis_client.set(f"rec:{target_id}", json.dumps(rec))
            
    os.makedirs(os.path.dirname(CONFIG["output_file"]), exist_ok=True)
    with open(CONFIG["output_file"], "w") as f:
        for rec in recs:
            f.write(json.dumps(rec) + "\n")

    print(f"✅ Completed. Stored {len(recs)} recommendations to Redis and {CONFIG['output_file']}")

# === Inference Utility for Batch Prediction ===
def predict_batch(model, id_mapper, batch: dict, device=torch.device("cpu")) -> np.ndarray:
    tgt = torch.tensor([id_mapper.item_to_index[i] for i in batch["target_items"]], device=device)
    ctx = torch.tensor([id_mapper.item_to_index[i] for i in batch["context_items"]], device=device)
    with torch.no_grad():
        return model(tgt, ctx).cpu().numpy()

# === MAIN ===
if __name__ == "__main__":
    compute_recommendations()

2025/06/29 09:04:04 WARNING mlflow.pytorch: Stored model version '2.6.0+cu124' does not match installed PyTorch version '2.7.1+cu126'
/home/duong/Documents/datn1/.venv/lib/python3.11/site-packages/torch/serialization.py:1488: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


✅ Loaded model v1 (run_id=aac89aa42a3c447eac1294546df933e6) and IDMapper.


Processing:   0%|          | 0/17 [00:00<?, ?it/s]

✅ Completed. Stored 4143 recommendations to Redis and ../../data/batch_recs.jsonl


In [6]:
import redis
import json

# Cấu hình Redis (đảm bảo giống CONFIG trong script gốc)
redis_client = redis.Redis(
    host="localhost",        # Hoặc thay bằng giá trị từ CONFIG["redis_host"]
    port=6379,               # CONFIG["redis_port"]
    db=0,                    # CONFIG["redis_db"]
    decode_responses=True,   # Đảm bảo trả về string thay vì bytes
    password="123456"    # Nếu có auth
)

# Item ID cần truy vấn
item_id = "B0002YV94U"
key = f"rec:{item_id}"

# Lấy dữ liệu từ Redis
rec_json = redis_client.get(key)

if rec_json:
    rec_data = json.loads(rec_json)
    print("✅ Recommendation found:")
    print(json.dumps(rec_data, indent=2))
else:
    print(f"❌ No recommendation found for item {item_id}")


✅ Recommendation found:
{
  "target_item": "B0002YV94U",
  "rec_item_ids": [
    "B087GTFML5",
    "B087GY3RJS",
    "B08KXGHX9V",
    "B00K8A08YU",
    "B0C3GL76CN",
    "B00DW1JT3I",
    "B00K5OLKCI",
    "B00U5MVOX0",
    "B0CH269DQ5",
    "B00LDX3OFQ"
  ],
  "rec_scores": [
    0.6255476474761963,
    0.6213389039039612,
    0.616352379322052,
    0.6159809231758118,
    0.6147825121879578,
    0.6140996813774109,
    0.6071546673774719,
    0.603735089302063,
    0.6013337969779968,
    0.6004250645637512
  ]
}


## Load popular item to Redis

In [ ]:
import pandas as pd
import redis
from sqlalchemy import create_engine

from dotenv import load_dotenv
import os

dotenv_path = os.path.abspath("../../.env")
load_dotenv(dotenv_path)

POSTGRES_URI = os.environ["POSTGRES_URI_OLTP"]
conn_str = POSTGRES_URI
table = "public.reviews"

# Redis config
redis_client = redis.Redis(
    host="localhost",
    port=6379,
    db=0,
    decode_responses=True,
    password="123456"
)

# 1. Load từ PostgreSQL
engine = create_engine(conn_str)
query = f"""
    SELECT 
        parent_asin,
        rating
    FROM {table}
    WHERE parent_asin IS NOT NULL
"""
df = pd.read_sql(query, engine)

# 2. Tính toán thống kê
agg_df = (
    df.groupby("parent_asin")
    .agg(rating_count=("rating", "count"), rating_avg=("rating", "mean"))
    .reset_index()
)

# 3. Tính popularity score
agg_df["score"] = agg_df["rating_count"] * (agg_df["rating_avg"] - 3.0)

# 4. Lấy top phổ biến theo score
TOP_K = 500
top_df = agg_df.sort_values("score", ascending=False).head(TOP_K)

# ✅ In ra để kiểm tra trước khi lưu
print("🔍 Top popular parent_asin (score-based):")
print(top_df[["parent_asin", "rating_count", "rating_avg", "score"]])

# 5. Ghi vào Redis nếu OK
redis_key = "popular_parent_asin_score"
redis_client.delete(redis_key)

for _, row in top_df.iterrows():
    redis_client.zadd(redis_key, {row["parent_asin"]: float(row["score"])})

print(f"\n✅ Đã lưu {len(top_df)} popular parent_asin vào Redis key: {redis_key}")


🔍 Top popular parent_asin (score-based):
     parent_asin  rating_count  rating_avg  score
3934  B0BW3QTWJJ           438    4.780822  780.0
2559  B07C4NGT17           226    4.867257  422.0
801   B0054TRQA4           216    4.750000  378.0
1223  B00D8STBHY           202    4.816832  367.0
3589  B09QPXVW35           166    4.885542  313.0
...          ...           ...         ...    ...
91    B000067NXE            45    4.333333   60.0
3290  B08PMPDGXM            36    4.638889   59.0
311   B000NV7L0I            42    4.404762   59.0
103   B000088UPW            56    4.053571   59.0
1451  B00IL7IFP6            33    4.787879   59.0

[500 rows x 4 columns]

✅ Đã lưu 500 popular parent_asin vào Redis key: popular_parent_asin_score


In [11]:
import redis

# Kết nối Redis
redis_client = redis.Redis(
    host="localhost",
    port=6379,
    db=0,
    decode_responses=True,
    password="123456"
)

# Redis key bạn đã lưu trước đó
redis_key = "popular_parent_asin_score"

# Truy vấn top 10 phổ biến nhất (score cao nhất)
top_items = redis_client.zrevrange(redis_key, 0, 9, withscores=True)

# In ra kết quả
print("🔥 Top 10 popular parent_asin from Redis:")
for rank, (asin, score) in enumerate(top_items, start=1):
    print(f"{rank:2d}. ASIN: {asin} | Score: {score:.2f}")


🔥 Top 10 popular parent_asin from Redis:
 1. ASIN: B0BW3QTWJJ | Score: 780.00
 2. ASIN: B07C4NGT17 | Score: 422.00
 3. ASIN: B0054TRQA4 | Score: 378.00
 4. ASIN: B00D8STBHY | Score: 367.00
 5. ASIN: B09QPXVW35 | Score: 313.00
 6. ASIN: B07N29HQMN | Score: 257.00
 7. ASIN: B00FZMDAO6 | Score: 248.00
 8. ASIN: B0BG94QRLZ | Score: 241.00
 9. ASIN: B09PH8LV57 | Score: 238.00
10. ASIN: B004S8F7QM | Score: 238.00
